In [26]:
# Shortest Job First (SJF) Scheduling Algorithm
# Process format: {'processor': 'P1', 'arrival_time': 0, 'CPU_time': 3}

def sjf(process_list):
    # Sort process list by arrival time first, then by CPU time
    process_list.sort(key=lambda x: (x['arrival_time'], x['CPU_time']))

    t = 0  # Current time
    gantt = []  # Gantt chart
    completed = {}  # Store details of completed processes
    total_turnaround_time = 0  # Sum of turnaround times for all processes

    # Use "while" statement until all processes are completed
    while process_list:
        available = [] #available processes

        # Gather processes that have arrived and are available at current time
        for process in process_list:
            if process['arrival_time'] <= t:
                available.append(process)

        # If no process has arrived (empty list), CPU remains idle
        if available == []:
            t += 1  # increment time
            gantt.append("IDLE")  # CPU is idle
        else:
            # Sort available processes by CPU total time, then choose process with shortest CPU total time
            available.sort(key=lambda x: x['CPU_time'])
            process = available[0]

            # simplify details from process
            burst_time = process['CPU_time']
            pid = process['processor']
            arrival_time = process['arrival_time']

            # Execute the process and update time
            t += burst_time
            gantt.append(pid)  # Add process to Gantt chart

            # Calculate completion time, turnaround time, and waiting time
            ct = t  # Completion time
            tt = ct - arrival_time  # Turnaround time (Completion Time - Arrival Time)
            wt = tt - burst_time  # Waiting time (Turnaround Time - Burst Time)

            # Remove the process from the process list ***important***
            process_list.remove(process)

            # Store the completed process results
            completed[pid] = {'Completion Time': ct, 'Turnaround Time': tt, 'Waiting Time': wt}

            # Add to total turnaround time for calculating average
            total_turnaround_time += tt


    # Calculate average turnaround time
    avg_turnaround_time = total_turnaround_time / len(completed)

    # Print results at the end
    print("\n*** SJF Scheduling Results ***")
    print(f"Gantt Chart: {gantt}")
    print("\nProcess Completion Details:")
    for pid, times in completed.items():
        print(f"{pid}: Completion Time = {times['Completion Time']}, Turnaround Time = {times['Turnaround Time']}, Waiting Time = {times['Waiting Time']}")

    print(f"\nAverage Turnaround Time: {avg_turnaround_time:.2f} units")

# Example process list
process_list = [
    {'processor': 'P1', 'arrival_time': 0, 'CPU_time': 3},
    {'processor': 'P2', 'arrival_time': 2, 'CPU_time': 6},
    {'processor': 'P3', 'arrival_time': 4, 'CPU_time': 4},
    {'processor': 'P4', 'arrival_time': 6, 'CPU_time': 5},
    {'processor': 'P5', 'arrival_time': 8, 'CPU_time': 2}
]

# Run the SJF scheduling salgorithm
sjf(process_list)



*** SJF Scheduling Results ***
Gantt Chart: ['P1', 'P2', 'P5', 'P3', 'P4']

Process Completion Details:
P1: Completion Time = 3, Turnaround Time = 3, Waiting Time = 0
P2: Completion Time = 9, Turnaround Time = 7, Waiting Time = 1
P5: Completion Time = 11, Turnaround Time = 3, Waiting Time = 1
P3: Completion Time = 15, Turnaround Time = 11, Waiting Time = 7
P4: Completion Time = 20, Turnaround Time = 14, Waiting Time = 9

Average Turnaround Time: 7.60 units


In [27]:
# Shortest Remaining Time (SRT) Scheduling Algorithm
# Process format: {'processor': 'P1', 'arrival_time': 0, 'CPU_time': 3}

def srt_scheduler(process_list):
    # Sort process list by arrival time first
    process_list.sort(key=lambda x: x['arrival_time'])

    t = 0  # Current time
    gantt = []  # Gantt chart
    completed = {}  # Store details of completed processes
    total_turnaround_time = 0  # Sum of turnaround times for all processes

    remaining_times = {p['processor']: p['CPU_time'] for p in process_list}  # Remaining times of processes
    ready_queue = []  # Queue for processes that are ready to run (not being used yet)
    current_process = None  # Tracks the current running process


    # Use "while" statement until all processes are completed
    while process_list or ready_queue:
        # Add processes to ready queue when they arrive
        while process_list and process_list[0]['arrival_time'] <= t:
            ready_queue.append(process_list.pop(0))

        # If no process has arrived (empty list), CPU remains idle
        if ready_queue == []:
            t += 1  # increment time
            if gantt == [] or gantt[-1] != "IDLE":
                gantt.append("IDLE")  # CPU is idle
        else:
            # Sort ready queue by remaining CPU time (preemptive), then choose process with shortest CPU total time
            ready_queue.sort(key=lambda x: remaining_times[x['processor']])
            next_process = ready_queue[0]
            pid = next_process['processor']

            # If process changes (b/c of shorest cpu total time), add it to the Gantt chart
            if current_process != pid:
                gantt.append(pid)  # Add process to the Gantt chart when there's a switch
                current_process = pid

            # Execute the process for 1 time unit (preemptive, so we execute step-by-step)
            remaining_times[pid] -= 1
            t += 1  # increment time

            # Check if remaining time is 0.
            if remaining_times[pid] == 0:
                # Calculate completion time, turnaround time, and waiting time
                ct = t  # Completion time
                tt = ct - next_process['arrival_time']  # Turnaround time (Completion Time - Arrival Time)
                wt = tt - next_process['CPU_time']  # Waiting time (Turnaround Time - Burst Time)

                # Store the completed process results
                completed[pid] = {'Completion Time': ct, 'Turnaround Time': tt, 'Waiting Time': wt}

                # Remove the process from the ready queue
                ready_queue.pop(0)

                # Add to total turnaround time for calculating average
                total_turnaround_time += tt

    # Calculate average turnaround time
    avg_turnaround_time = total_turnaround_time / len(completed)

    # Print results at the end
    print("\n*** SRT Scheduling Results ***")
    print(f"Gantt Chart: {gantt}")
    print("\nProcess Completion Details:")
    for pid, times in completed.items():
        print(f"{pid}: Completion Time = {times['Completion Time']}, Turnaround Time = {times['Turnaround Time']}, Waiting Time = {times['Waiting Time']}")

    print(f"\nAverage Turnaround Time: {avg_turnaround_time:.2f} units")


# Example process list
process_list = [
    {'processor': 'P1', 'arrival_time': 0, 'CPU_time': 3},
    {'processor': 'P2', 'arrival_time': 2, 'CPU_time': 6},
    {'processor': 'P3', 'arrival_time': 4, 'CPU_time': 4},
    {'processor': 'P4', 'arrival_time': 6, 'CPU_time': 5},
    {'processor': 'P5', 'arrival_time': 8, 'CPU_time': 2}
]

# Run the SRT scheduler
srt_scheduler(process_list)



*** SRT Scheduling Results ***
Gantt Chart: ['P1', 'P2', 'P3', 'P5', 'P2', 'P4']

Process Completion Details:
P1: Completion Time = 3, Turnaround Time = 3, Waiting Time = 0
P3: Completion Time = 8, Turnaround Time = 4, Waiting Time = 0
P5: Completion Time = 10, Turnaround Time = 2, Waiting Time = 0
P2: Completion Time = 15, Turnaround Time = 13, Waiting Time = 7
P4: Completion Time = 20, Turnaround Time = 14, Waiting Time = 9

Average Turnaround Time: 7.20 units
